<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Ensure the right version of Tensorflow is installed.
#!pip freeze | grep tensorflow==2.1

In [3]:
# change these to try this notebook out
BUCKET = 'teste1espm'
PROJECT = 'teste1espm'
REGION = 'us-central1'

In [4]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [5]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [7]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-2599641534368510197,309791
1,-4701948054522374613,340518
2,-1808845555948931465,355725
3,-1195438672706281328,362513
4,6392072535155213407,323758


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [8]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13356 examples in the train dataset and 3241 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [9]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.818490,True,33,1,37,3095933535584005890
1,8.000575,True,25,1,39,3095933535584005890
2,6.499227,True,21,1,40,3095933535584005890
3,8.999270,True,43,1,39,3095933535584005890
4,7.061406,False,26,1,42,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [10]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13348.000000,13356.0,13356.0,13271.0,13356.0
mean,7.244753,27.472222,1.034441,38.63198,321343057937640768.0
std,1.291459,6.13965,0.194295,2.554848,5218555972309268480.0
min,0.608476,13.0,1.0,18.0,-9183605629983195136.0
25%,6.600640,23.0,1.0,38.0,-3340563182046119936.0
50%,7.317142,27.0,1.0,39.0,-328012383083104832.0
75%,8.062305,32.0,1.0,40.0,5896567601480310784.0
max,13.562838,50.0,4.0,47.0,8599690069971956736.0


In [11]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  #twins_etc = dict(zip([1,2,3,4,5],['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  #df['plurality'].replace(twins_etc, inplace=True)

  twins_etc = {
    1: 'Single(1)', 
    2: 'Twins(2)', 
    3: 'Triplets(3)', 
    4: 'Quadruplets(4)', 
    5: 'Quintuplets(5)'
  }

  # Supondo que o DataFrame df já existe e tem uma coluna 'plurality'
  # Substituindo os valores na coluna 'plurality'
  df['plurality'] = df['plurality'].map(twins_etc)



  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [12]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.818490,True,33,Single(1),37,3095933535584005890
1,8.000575,True,25,Single(1),39,3095933535584005890
2,6.499227,True,21,Single(1),40,3095933535584005890
3,8.999270,True,43,Single(1),39,3095933535584005890
4,7.061406,False,26,Single(1),42,3095933535584005890


In [13]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13351,7.438397,Unknown,28,Single(1),40,-774501970389208065
13352,6.567571,Unknown,35,Single(1),39,-774501970389208065
13353,7.142977,Unknown,38,Single(1),36,-774501970389208065
13354,7.687519,Unknown,25,Single(1),38,-774501970389208065
13355,8.090965,Unknown,34,Single(1),39,-774501970389208065


In [14]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26528.000000,26528.0,26528.0,26528.0
mean,7.244392,27.473613,38.637289,325489780202930432.0
std,1.291487,6.139031,2.537464,5215285658833999872.0
min,0.608476,13.0,18.0,-9183605629983195136.0
25%,6.602294,23.0,38.0,-3340563182046119936.0
50%,7.318245,27.0,39.0,-328012383083104832.0
75%,8.062305,32.0,40.0,5896567601480310784.0
max,13.562838,50.0,47.0,8599690069971956736.0


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [15]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [16]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6436 eval.csv
  26528 train.csv
  32964 total
==> eval.csv <==
8.18796841068,True,29,Single(1),39,1569531340167098963
6.97542596968,False,23,Single(1),39,-6141045177192779423
7.62578964258,False,34,Single(1),40,-1866590652208008467
7.9035720927,False,27,Single(1),38,-1866590652208008467
7.3413933246,True,32,Single(1),39,-7517141034410775575
5.37486994756,False,24,Single(1),37,6365946696709051755
8.000575487979999,True,20,Single(1),39,-7517141034410775575
7.6720867176,True,29,Single(1),39,270792696282171059
8.75014717878,True,26,Single(1),39,3182182455926341111
7.3193470984,True,19,Single(1),38,3182182455926341111

==> train.csv <==
8.81849048,True,33,Single(1),37,3095933535584005890
8.000575487979999,True,25,Single(1),39,3095933535584005890
6.4992274837599995,True,21,Single(1),40,3095933535584005890
8.99926953484,True,43,Single(1),39,3095933535584005890
7.06140625186,False,26,Single(1),42,3095933535584005890
8.811876612139999,True,29,Single(1),44,3095933535584005890
7.2201390805,Tru

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License